**[Kaggle Machine Learning Course Home Page](kaggle.com/learn/machine-learning).**

---

# Selecionando dados para Modelagem
Se o conjunto de dados possui muitas variáveis, usar todas elas pode criar um modelo mais complexo que o necessário e gerar um Sobreajuste (overfitting)! Porém se selecionarmos muito poucas, o modelo pode ficar pobre em predição e sofre de um Subajuste (underfitting).

Vamos tentar selecionar apenas algumas variáveis, com base na intuição. Técnicas estatísticas de seleção de variáveis são preferíveis, como veremos mais pra frente

Primeiros vamos carregar o dataset e ver quantas colunas temos com o código a seguir.


In [1]:
import pandas as pd

melbourne_file_path = 'melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

Como é bastante comum, este dataset possui alguns valores vazios (marcados pelo Pandas como NaN - vamos descartar esses dados por enquanto!)

In [2]:
# dropna: descarta valores não preenchidos
melbourne_data = melbourne_data.dropna()

Como viram na parte de Pandas, podemos selecionar colunas de um DataFrame de duas formas:

1. Notação de ponto (DataFrame.Coluna);
2. Notação de lista de colunas (DataFrame['Coluna']).

## Selecionando a variável alvo

Ambas as notações retornam o tipo de dados **Series**, que pode ser entendido como um DataFrame de uma única coluna.

A seguir, usamos a notação de ponto para selecionar a variável que queremos prever, chamada de ***variável dependente ou variável alvo***. Por convenção, normalmente chamamos a essa variável de **y** (vamos respeitar as tradições!). Então o código para colocarmos a variável Preço (Price) como variável dependente ou alvo é:

In [3]:
y = melbourne_data.Price

# Escolhendo "Features"
As colunas que escolheremos para fazer parte do nosso modelo são as chamadas "atributos" ou "features". Como dissemos, para evitar um Sobreajuste do modelo, nem sempre é razoável utilizar todas as colunas de seu dataset como atributos. 

Existem técnicas estatísticas que permitem selecionar os melhores atributos para uma variável alvo, mas por ora vamos escolher algumas variáveis na mão mesmo, construindo uma lista como a do exemplo a seguir:

In [4]:
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']

Também por convenção, chamamos o conjunto de features de **X**

In [5]:
X = melbourne_data[melbourne_features]

Antes de construir um modelo, é sempre bom revisar os dados, pois podemos achar problemas que não foram previstos na carga! Podemos obter uma descrição resumida dos dados com `describe` e ver as primeiras linhas do DataFrame com `head`.

In [6]:
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [7]:
X.head()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954


## Conjunto de treino e conjunto de testes
Ainda antes de construir o modelo, precisamos dividir nosso conjunto de dados entre os dados que serão utilizados para o modelo aprender padrões e os dados que utilizaremos para avaliar os resultados. 

Isso deve ser feito porque se testarmos o modelo com os mesmos dados que utilizamos para fazê-lo aprender, estaremos subestimando o desempenho dele. Os dados de teste devem ser idealmente próximos a dados reais externos que o modelo não conhece, só assim saberemos se ele realmente é útil na prática, livre de possíveis sobreajustes que possam ocorrer no conjunto de treinamento.

Normalmente, também por convenção (sempre ela!), o conjunto de treinamento corresponde a 2/3 do conjunto original, enquanto o de teste corresponde a 1/3 do total. Essa proporção pode variar, mas em geral é bom dedicarmos a maior parte de nosso conjunto de dados para treinamento.

Esta divisão pode ser feita com o auxílio da função `train_test_split` da biblioteca scikit-learn, que vermos com mais detalhes a seguir. Na aplicação desta função abaixo, vemos que definimos o conjunto de testes como 1/3 do total (`test_size=0.33`). 

O uso do parâmetro `random_state=42` é para que possamos fazer sempre a mesma divisão toda vez que rodarmos esse código, evitando divisões randômicas a cada rodada. Isso é importante para manter este código reproduzível por todos, sem resultados diferentes para cada pessoa que tente executá-lo.

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

---
# Construindo seu modelo

Vamos usar a super biblioteca **scikit-learn** para criar nossos modelos. No código, chamamos essa bibliteca de **sklearn**, como vai ver na amostra de código a seguir. Scikit-learn é de longe a biblioteca mais popular para criar modelos em Python a partir de dados armazenados em DataFrames.

**Lembre-se** sempre dos 4 passos a seguir, pois são usados para criar qualquer tipo de modelo no Scikit-learn:

* **Definição:** Qual o tipo de modelo você vai usar? Uma regressão linear? Uma árvore de decisão? Outro tipo de modelo mais complexo? Dependendo do modelo alguns parâmetros adicionais podem ser necessários.
* **Ajuste (Fit):** Captura os padrões nos dados. É a etapa de treinamento, o centro de qualquer modelo. Lembre de utilizar aqui apenas seus **dados de treinamento**.
* **Previsão (Predict):** Prevê o resultado da variável alvo com base no que foi ajustado (treinado) na etapa anterior
* **Avaliação**: Determina a acurácia do modelo. Lembre-se de utilizar aqui apenas os **dados de teste**.

Segue exemplo de definição de uma regressão linear no scikit-learn e ajuste utilizando os features e a variável alvo definida (os dois primeiros passos descritos acima).

In [9]:
from sklearn.linear_model import LinearRegression

# Definir o modelo (Regressão Linear)
melbourne_model = LinearRegression()

# Ajustar o modelo (treinamento)
melbourne_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Acabamos de ajustar uma reta nos dados de treinamento, vamos ver os coeficientes desse modelo.

In [20]:
[(X_train.columns[i], melbourne_model.coef_[i]) for i in range(len(X_train.columns))]

[('Rooms', 263580.59846338053),
 ('Bathroom', 239093.58097595171),
 ('Landsize', 15.518265321077649),
 ('Lattitude', -1621514.973100259),
 ('Longtitude', 638031.4890833759)]

Com isso podemos ver que número de quartos (Rooms), Banheiros (Bathroom), Tamanho do terreno (Landsize) e e Longitude aumentam o preço do imóvel. Já latitude diminui o preço do imóvel (provavelmente devido à localização do bairro onde o imóvel está localizado). No entanto, não podemos ver exatamente qual dessas variáveis mais influencia no preço porque os dados estão desnormalizados (deixaremos a normalização como exercício). 

Agora que temos um modelo ajustado pelo conjunto de treino, podemos usá-lo para fazer previsões nos dados de teste ou em dados externos ao dataset original!

In [21]:
print("Prevendo o valor para as 5 primeiras casas do conjunto de teste:")
print(X_test.head())
print("Os preços previstos são:")
print(melbourne_model.predict(X_test.head()))

Prevendo o valor para as 5 primeiras casas do conjunto de teste:
       Rooms  Bathroom  Landsize  Lattitude  Longtitude
5379       2       2.0       0.0  -37.81660   145.00360
11334      3       2.0     801.0  -37.98596   145.13187
2259       2       1.0     553.0  -37.88020   144.99420
3833       2       1.0     196.0  -37.87960   145.08760
6681       4       2.0     417.0  -37.81970   144.89690
Os preços previstos são:
[  946099.54479712  1578570.34873167   812718.42083552   865797.63221243
  1416680.59489417]


## Avaliando seu modelo
Existem diversas métricas para avaliar um modelo de aprendizagem de máquina. Para regressão linear, uma métrica comum é o **Coeficiente de Deteminação (R<sup>2</sup>)**, que pode ser entendido como a porcentagem de variação da variável alvo que é explicada pelos features selecionados.

In [22]:
print("O coeficiente de determinação (R^2) é:")
print(melbourne_model.score(X_test, y_test))

O coeficiente de determinação (R^2) é:
0.383421165989


Ou seja, as cinco variáveis selecionadas (Rooms, Bathroom, Landsize, Lattitude e Longtitude) explicam apenas cerca de 38% da variação do Preço dos imóveis neste conjunto de dados.